### Preparing data

In [1]:
import os
import random
import shutil
import pandas as pd
import cv2
import numpy as np
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import regularizers
from keras.layers import LeakyReLU, ELU

Using TensorFlow backend.


In [2]:
class DataGenerator:
	DATASET_DIR = "C:\\Users\\ASUS\\Documents\\Proyectos\\pokedex\\pokemon"

	def __init__(self):
		self.category_dirs = os.listdir(DataGenerator.DATASET_DIR)
		self.data = pd.DataFrame({}, columns = ['label', 'image']) 
		self.data_train = pd.DataFrame({}, columns = ['label', 'image']) 
		self.data_test = pd.DataFrame({}, columns = ['label', 'image']) 
        
	def generate(self):
		for category in self.category_dirs:
			category_path = os.path.join(DataGenerator.DATASET_DIR, category)
			if not os.path.isdir(category_path):
				print("no es directorio valido")
				continue
			images = os.listdir(category_path)
			bunch = pd.DataFrame({}, columns = ['label', 'image']) 
			for img in images:
				imgArr = cv2.imread("pokemon\\"+str(category)+"\\"+str(img))
				bunch = bunch.append({'label' : category.upper() , 'image' : imgArr},ignore_index=True)
				#self.data = self.data.append({'label' : category.upper() , 'image' : imgArr},ignore_index=True)
			msk = np.random.rand(len(bunch)) < 0.8
			self.data_train = self.data_train.append(bunch[msk])
			self.data_test = self.data_test.append(bunch[~msk])

In [ ]:
def createFolderIfNotExists(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        #print("Directory data_generated already exists")
        return

In [32]:
DATASET_DIR = "C:\\Users\\ASUS\\Documents\\Proyectos\\pokedex\\pokemon"
numberAnomalies = 0
for category in os.listdir(DATASET_DIR):
    category_path = os.path.join(DataGenerator.DATASET_DIR, category)
    if not os.path.isdir(category_path):
        print("no es directorio valido")
        continue
    images = os.listdir(category_path)
    totalImgs = len(images)
    counter = 0
    path = "data"
    createFolderIfNotExists(path)
    for img in images:
        tempPath = path
        #print(counter / totalImgs)
        if(counter / totalImgs < 0.7):
            tempPath += "\\train"
        elif(counter / totalImgs < 0.85):
            tempPath += "\\test"
        else:
            tempPath += "\\validation"
        createFolderIfNotExists(tempPath)
        tempPath += "\\"+str(category)
        createFolderIfNotExists(tempPath)
        tempPath += "\\"+str(img)
        imgArr = cv2.imread("pokemon\\"+str(category)+"\\"+str(img))
        try:
            cv2.imwrite(tempPath, imgArr) 
        except:
            numberAnomalies += 1
                
        counter += 1
print("numberAnomalies: "+str(numberAnomalies))

numberAnomalies: 33


In [2]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

In [3]:
train_it = datagen.flow_from_directory('data\\train\\', class_mode='categorical', batch_size=32)
val_it = datagen.flow_from_directory('data\\validation\\', class_mode='categorical', batch_size=32)
test_it = datagen.flow_from_directory('data\\test\\', class_mode='categorical', batch_size=32)

Found 7529 images belonging to 149 classes.
Found 1524 images belonging to 149 classes.
Found 1604 images belonging to 149 classes.


In [54]:
bx,by = train_it.next()
np.shape(bx)

(64, 256, 256, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3,3),kernel_regularizer=regularizers.l2(1e-4),input_shape=(256, 256, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3),kernel_regularizer=regularizers.l2(1e-4)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
 
model.add(Conv2D(64, (3,3),kernel_regularizer=regularizers.l2(1e-4)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3),kernel_regularizer=regularizers.l2(1e-4)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(149, activation='sigmoid'))
 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 254, 254, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 254, 254, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 252, 252, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 252, 252, 32)      128       
_________________________________________________________________
max_

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=8,epochs=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
16/16 [==============================] - 12s 746ms/step - loss: 11.7789 - acc: 0.0117 - val_loss: 13.8669 - val_acc: 0.0156
Epoch 2/50
16/16 [==============================] - 10s 642ms/step - loss: 10.2995 - acc: 0.0078 - val_loss: 13.5657 - val_acc: 0.0195
Epoch 3/50
16/16 [==============================] - 11s 664ms/step - loss: 10.1953 - acc: 0.0137 - val_loss: 10.3432 - val_acc: 0.0000e+00
Epoch 4/50
16/16 [==============================] - 11s 658ms/step - loss: 9.2284 - acc: 0.0039 - val_loss: 9.6452 - val_acc: 0.0000e+00
Epoch 5/50
16/16 [==============================] - 10s 634ms/step - loss: 9.8731 - acc: 0.0039 - val_loss: 10.5221 - val_acc: 0.0000e+00
Epoch 6/50
16/16 [==============================] - 11s 681ms/step - loss: 9.2849 - acc: 0.0098 - val_loss: 12.8470 - val_acc: 0.0164
Epoch 7/50
16/16 [==============================] - 11s 676ms/step - loss: 9.1984 - acc: 0.0156 - val_loss: 8.6334 - val_acc: 0.0039
E